# Loss Functions

Changes in this version
-  Add different loss functions

In [3]:
import numpy as np
np.random.seed(0) # in notebooks, this needs to be present in the cell where the random is being called
np.set_printoptions(precision=4)

class Activation:
    def __init__(self):
        pass
    def __repr__(self):
        pass
    def forward(self, inputs):
        pass
    def backward(self, outputs):
        pass
    def __call__(self, arg):
        return self.forward(arg)

# ReLU function
class Act_Linear(Activation):
    def forward(self, inputs):
        return inputs

    def backward(self, outputs):
        return 1

    def __repr__(self):
        return "Act_Linear"

class Act_ReLU(Activation):
    def forward(self, inputs):
        return np.maximum(0, inputs)

    def backward(self, outputs):
        return np.where(outputs > 0, 1, 0)
    
    def __repr__(self):
        return "Act_ReLU"

class Act_Tanh(Activation):
    def forward(self, inputs):
        return np.tanh(inputs)

    def backward(self, outputs):
        # dy/dx = 1-y**2
        return (1 - outputs**2)
    
    def __repr__(self):
        return "Act_Tanh"

class Act_Sigmoid(Activation):
    def forward(self, inputs):
        return 1/(1+np.exp(-inputs))

    def backward(self, outputs):
        # dy/dx = y*(1-y)
        return outputs * (1-outputs)
    
    def __repr__(self):
        return "Act_Sigmoid"

class Act_Softmax(Activation):
    def forward(self, inputs):
        exp = np.exp(inputs)
        return exp / np.sum(exp)

    def backward(self, outputs):
        # TODO: y_k * (1 - y_i) when i = k
        #       y_k * (  - y_i) when i != k
        pass
    
    def __repr__(self):
        return "Act_Softmax"

class Layer:
    def __init__(self, n_inputs, n_neurons, activation_fn, weights=None, biases=None):
        if activation_fn is None:
            activation_fn = Act_Linear

        if activation_fn is Act_Softmax:
            raise Exception("Softmax is not supported as an activation function, use it after the output")
            
        if weights is None:
            self.weights = 0.1 * np.random.randn(n_neurons, n_inputs) # multiplying with 0.1 to keep the range within (-1, 0, 1)
        else:
            self.weights = weights # used to test the correction of my code

        if biases is None:
            self.biases = np.zeros((1, n_neurons))
        else:
            self.biases = biases
        self.activation = activation_fn()  # new code - initialise the activation class
        self.inputs = []
        self.grad_act = []
        self.grad_new = []
        self.grad_biases = []

    def forward(self, inputs):
        # modified to execute the activation forward code
        self.inputs = inputs
        output_raw = np.dot(self.inputs, self.weights.T) + self.biases
        self.output = self.activation.forward(output_raw)
        return self.output

    def backward(self, prev_grad):
        self.grad_act = self.activation.backward(self.output) # gradient of the activation fn
        self.grad_new = np.multiply(prev_grad, self.grad_act)
        self.grad_weights = np.dot(self.grad_new.T, self.inputs)
        self.grad_biases = np.sum(self.grad_new, axis=0, keepdims=True)
        return np.dot(self.grad_new, self.weights)

    def __call__(self, arg):
        return self.forward(arg)

    def __repr__(self):
        return f"Layer(n_inp={self.weights.shape[1]},\
        n_neurons={self.weights.shape[0]},\
        activation_fn={self.activation.__repr__()})"

# Linea
class LinearModel:
    def __init__(self, *args):
        self.layers = []
        for arg in args:
            self.layers.append(arg)

    def __call__(self, arg):
        return self.forward(arg)

    def forward(self, arg):
        out = arg
        for layer in self.layers:
            out = layer(out)
        return out

    def backward(self, grad):
        # TODO: how to handle different dimension data for the prev gradient
        for layer in self.layers[::-1]:
            grad = layer.backward(grad)
        return grad

    def __repr__(self):
        head = "LinearModel(\n"
        tail = ")"
        body = ""
        for layer in self.layers:
            body += layer.__repr__() + "\n"
        return head + body + tail